# Autores:
### Fernando Fabrizzio Ramírez Flores A01551949
### Andres Piñones Besnier A01570150
### Edith Paulina Benvenuto Valerdi A00828840
### Joaquin Arturo Beltrán López A00827019

* Realiza la siguiente simulación: 
  * Inicializa las posiciones iniciales de las K cajas. Todas las cajas están a nivel de piso, 
es decir, no hay pilas de cajas. 
  * Todos los agentes empiezan en posición aleatorias vacías. 
  * Se ejecuta el tiempo máximo establecido. 
  * Se pueden acumular máximo 5 cajas por cada posición.
* Deberás recopilar la siguiente información durante la ejecución: 
  * Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas. 
  * Número de movimientos realizados por todos los robots. 
  * Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como 
la cantidad de movimientos realizados. ¿Cómo sería? Descríbela. 


### Imports

In [2]:
!pip install mesa

# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Se importa MultiGrid pues pueden existir varias entidades en una celda
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import math
import random

     |████████████████████████████████| 668 kB 27.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


# Agentes

In [3]:
def get_grid(model):
  #Se obtiene grid del modelo, se itera sobre la misma, se leen sus valores y se asigna un valor a cada celda para representación gráfica
  grid = np.zeros((model.grid.width, model.grid.height))
  for cell in model.grid.coord_iter():
    cell_content, x, y = cell
    for content in cell_content:
      if isinstance(content,Shelf):
        grid[x][y] = content.stacks 
      elif isinstance(content,Robot):
        grid[x][y] =  9
      elif isinstance(content,Floor):
        if content.isBox:
          grid[x][y] = 7
        else:
          grid[x][y] = 8
      else:
        grid[x][y] = 6
  return grid


class Shelf(Agent):
  def __init__(self,unique_id, pos, model):
    super().__init__(unique_id,model)
    self.unique_id = unique_id
    self.pos = pos
    self.stacks = 0
    self.color = 10
    

class Floor(Agent):
  def __init__(self,unique_id, pos,isBox, model):
    super().__init__(unique_id,model)
    self.unique_id = unique_id
    self.pos = pos
    self.isBox = isBox


class Robot(Agent):
  def __init__(self,unique_id,pos,model):
    super().__init__(unique_id,model)
    self.hasBox = False
    self.inShelf = False
    self.pos = pos
    self.nextPosition = (0,0) #Despues de dejar la primera caja comenzará a moverse desde esa posicion
    self.direction = 1 #1 derecha -1 izquierda
  
  def nextMove(self, posToGo):
    #Esta funcion se mueve como en diagonal, directo a la posToGo
    if self.pos[0] > posToGo[0]: #El robot sube
      if self.pos[1] > posToGo[1]: #Robot va a la izquierda
        return (- 1, - 1)
      elif self.pos[1] < posToGo[1]: #Robot va a la derecha
        return ( -1 , 1)
      else: #Se queda en su misma pos en X
        return (-1, 0)
    elif self.pos[0] < posToGo[0]: #El robot baja
      if self.pos[1] > posToGo[1]: #El robot va a la izquierda
        return ( 1, - 1)
      elif self.pos[1] < posToGo[1]: #El robot va a la derecha
        return (1 , 1)
      else: #Se queda en su misma pos en X
        return (1, 0)
    else: #Se queda en su misma pos en Y
      if self.pos[1] > posToGo[1]: # El robot va a la izquierda
        return (0, - 1)
      elif self.pos[1] < posToGo[1]: #El robot va a la derecha
        return (0, 1)
      else: #El robot no se mueve
        return (0, 0)

  
  def move(self):
    #Movimiento como serpientes y escaleras
    auxPos = (0,0)
    if not self.pos == self.nextPosition:
      nextSteps = self.nextMove(self.nextPosition)
      auxPos = (self.pos[0] + nextSteps[0],self.pos[1] + nextSteps[1])
    else:
      #Llega a la posicion WxH
      if self.pos[1] == self.model.W -1 and self.pos[0] == self.model.H-1 and self.model.H % 2==1:
        self.nextPosition = (0,0)
        auxPos = (self.pos)
      elif self.pos[1] == 0 and self.pos[0] == self.model.H-1 and self.model.H % 2==0:
        self.nextPosition = (0,0)
        auxPos = (self.pos)
      #está en la ultima columna y avanza hacia la derecha
      elif self.pos[1] == self.model.W -1 and self.direction == 1:
        auxPos = (self.pos[0]+1 , self.model.W -1)
        self.direction = -1
      elif self.pos[1] == 0 and self.direction == -1:
        auxPos = (self.pos[0]+1,0)
        self.direction = 1
      #Avanzar si no se encuentra en las esquinas
      else:
        if self.direction == 1:
          auxPos = (self.pos[0],self.pos[1]+1)
        else:
          auxPos = (self.pos[0],self.pos[1]-1)
      self.nextPosition = auxPos 
    thisCell = self.model.grid.get_cell_list_contents([auxPos])
    robotsInCell = len([obj for obj in thisCell if isinstance(obj,Robot)])

    if robotsInCell >0:
      occupied = True
      count = 0
      availableMoves = self.model.grid.get_neighborhood(self.pos, True, False)
      random.shuffle(availableMoves)
      while count < len(availableMoves) and occupied:
        thisCell = self.model.grid.get_cell_list_contents(availableMoves[count])
        robotsInCell = len([obj for obj in thisCell if isinstance(obj,Robot)])
        if robotsInCell == 0:
          auxPos = availableMoves[count]
          occupied = False
        else: 
          count +=1     

    self.model.grid.move_agent(self, auxPos)
  

  def step(self):
    if self.hasBox:
      if self.pos == self.model.shelfPos: #Si está en el estante deja la caja si no está lleno
        thisCell = self.model.grid.get_cell_list_contents([self.pos])      
        shelf = [obj for obj in thisCell if isinstance(obj,Shelf)]
        shelf[0].stacks += 1
        self.model.shelves+=1
        self.hasBox = False
        if shelf[0].stacks == 5: #Si el estante esta lleno se cambia la posicion del estante
          if self.model.shelfPos[1]== self.model.W -1:
            self.model.shelfPos = (self.model.shelfPos[0]+1,0)
          else:
            self.model.shelfPos = (self.model.shelfPos[0],self.model.shelfPos[1]+1)
      else: #Tiene caja pero no ha llegado al estante
        nextSteps = self.nextMove(self.model.shelfPos)
        nextPos = (self.pos[0] + nextSteps[0],self.pos[1] + nextSteps[1])
        
        thisCell = self.model.grid.get_cell_list_contents(nextPos)
        hasRobot = len([obj for obj in thisCell if isinstance(obj,Robot)])
        if hasRobot == 0: #Si no hay ningun robot se mueve a esa posicion, sino se espera 
          self.model.grid.move_agent(self, nextPos)
    else: #Si no tiene caja sigue buscando
      thisCell = self.model.grid.get_cell_list_contents([self.pos])
      thisFloor = [obj for obj in thisCell if isinstance(obj,Floor)] 
      if thisFloor[0].isBox == 1 and not thisFloor[0].pos == model.shelfPos:
        thisFloor[0].isBox = False 
        self.hasBox = True
      else:
        self.move()


class StorageModel(Model):
  def __init__(self, W, H, boxes):
    self.finish = False
    self.nBoxes = boxes
    self.W = W
    self.H = H
    self.generations = 0
    self.nShelves = math.ceil(boxes / 5)
    self.shelves = 0
    self.shelfPos = (0,0)
    self.emptyFloor = W*H - boxes 

    self.grid = MultiGrid(H,W,False)
    self.schedule = SimultaneousActivation(self)

    #Boxes
    for i in range(self.nBoxes):
      emptyGridPos = self.grid.find_empty()
      newBox = Floor(i+self.nShelves,emptyGridPos,True,self)
      self.schedule.add(newBox)     
      self.grid.place_agent(newBox,emptyGridPos)

    #Empty floor
    for i in range(self.emptyFloor):
      emptyGridPos = self.grid.find_empty()
      newFloor = Floor(i*self.emptyFloor+1000,emptyGridPos,False,self)
      self.schedule.add(newFloor)     
      self.grid.place_agent(newFloor,emptyGridPos)

    self.paver()
    self.datacollector = DataCollector(model_reporters={"Grid": get_grid})

  def paver(self):
    #Shelves
    i = 0
    id = 0
    for j in range(self.nShelves):
      if j == self.W:
        i += 1
        j = 0
      newPos = (i,j)
      newFloor = Shelf( id , newPos, self)
      self.schedule.add(newFloor)
      self.grid.place_agent(newFloor,newPos)
      id += 1
      
    #Robots
    for i in range(5):
      emptyGridPos = (random.randint(0,self.W-1),random.randint(0,self.H-1))
      newRobot = Robot(i+100,emptyGridPos,self)
      self.schedule.add(newRobot)     
      self.grid.place_agent(newRobot,emptyGridPos)

  
  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()
    self.generations +=1
    if self.shelves == self.nBoxes:
      self.finish = True


# Ejecución

In [4]:
W = 10
H = 10
K = 15
EXEC_MAX_TIME = 0.05
model = StorageModel(W,H,K)
TIME_START = time.time() 

while((time.time() - TIME_START) < EXEC_MAX_TIME and not model.finish):
  model.step()

print('Tiempo tomado para apilar:', str(datetime.timedelta(seconds=(time.time() - TIME_START))))
print('Numero de pasos realizados:', model.generations)
if model.finish:
  print('Apiló todas las cajas') 
else:
  print('No le dió tiempo de apilar todas las cajas')
print('Porcentaje de cajas apiladas:', model.shelves/K*100,'%')


Tiempo tomado para apilar: 0:00:00.032412
Numero de pasos realizados: 126
Apiló todas las cajas
Porcentaje de cajas apiladas: 100.0 %


In [5]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [6]:
%%capture

fig, axs = plt.subplots(figsize=(6,6))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap ='ocean')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate,frames = len(all_grid))

In [7]:
anim